# **Analytics**

----------

In [1]:
DF_PATH = "../data/processed/1_preprocessed_df.pkl"
ROLES_PATH = "../data/raw/roles_short_names.csv"
FIG_DIR = "../reports/figures"

NA_STRING = "Not Specified"
TRANSPARENT_STRING = 'rbga(0,0,0,0)'

ROLES_COLS = ['DevType']
TECH_COLS = ['LanguageHaveWorkedWith',
             'LanguageWantToWorkWith', 
             'DatabaseHaveWorkedWith',
             'PlatformHaveWorkedWith',
             'PlatformWantToWorkWith',
             'WebframeHaveWorkedWith', 
             'WebframeWantToWorkWith',
             'MiscTechHaveWorkedWith',
             'MiscTechWantToWorkWith',
             'ToolsTechHaveWorkedWith', 
             'ToolsTechWantToWorkWith',
             "NEWCollabToolsHaveWorkedWith",
             "NEWCollabToolsWantToWorkWith"]

------------

## Load Data

In [2]:
# Libraries 
import pandas as pd 
import numpy as np 
import logging
import pickle 
import os 

# Visualisation Libraries
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from matplotlib import pyplot as plt

# Analysis & preproccessing Libraries
from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer , StandardScaler , RobustScaler , MinMaxScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import make_pipeline
from scipy.cluster.hierarchy import dendrogram , fcluster , linkage

------------

## Reading Data

In [3]:
raw_df = pd.read_pickle(DF_PATH)
roles_names = pd.read_csv(ROLES_PATH , sep=";")

In [4]:
df = raw_df.copy()
# checking the dataframe    
df.sample(4)

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
78722,78723,I am a student who is learning to code,"Student, full-time",United States of America,Florida,NaN,"Associate degree (A.A., A.S., etc.)",18 - 24 years,"[Other online resources (ex: videos, blogs, et...",2.0,...,18-24 years old,[Man],No,[Straight / Heterosexual],[Hispanic or Latino/a/x],[None of the above],[I have an anxiety disorder],Appropriate in length,Easy,NaN
56283,56284,I am a developer by profession,Employed full-time,South Africa,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Other online resources (ex: videos, blogs, et...",9.0,...,18-24 years old,[Man],No,[Straight / Heterosexual],[South Asian],[None of the above],[None of the above],Appropriate in length,Easy,52039.0
17164,17165,I code primarily as a hobby,Employed full-time,Italy,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Coding Bootcamp, Other online resources (ex: ...",10.0,...,25-34 years old,[Man],No,"[Straight / Heterosexual, Bisexual, Gay or Les...","[Multiracial, Middle Eastern, Black or of Afri...",[None of the above],[],Appropriate in length,Neither easy nor difficult,NaN
64282,64283,I am a student who is learning to code,"Student, part-time",India,NaN,NaN,Primary/elementary school,5 - 10 years,"[Other online resources (ex: videos, blogs, et...",5.0,...,Under 18 years old,[Man],No,[Straight / Heterosexual],[Southeast Asian],[None of the above],[None of the above],Appropriate in length,Neither easy nor difficult,NaN


In [5]:
# Take a look at the roles names
roles_names

,Original name,Short name
0,"Developer, back-end",Back-end dev
1,"Developer, full-stack",Full-stack dev
2,"Developer, front-end",Front-end dev
3,"Developer, desktop or enterprise applications",Desktop dev
4,"Developer, mobile",Mobile dev
5,DevOps specialist,DevOps
6,Database administrator,Database admin
7,Designer,Designer
8,System administrator,System admin
9,"Developer, embedded applications or devices",Embedded dev


------------

## One hot encoding 

In [6]:
encoded_dfs = {}
for col in ROLES_COLS + TECH_COLS:
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]), 
                              columns = binarizer.classes_ , 
                              index = df[col].index)
    encoded_dfs[col] = encoded_df

In [7]:
# merge 1-hot encoded 
df = pd.concat(encoded_dfs , axis = 1)
df

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
1                       0                        0   
2                       0                        0   
3                       0                        0   
4                       0                        0   
...                   ...                      ...   
83434                   0                        0   
83435                   0                        0   
83436                   0                        0   
83437                   0                        0   
83438                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
1                                                 0                      0   
2                                                 0                      0   
3                                                 0                      0   
4                                                 0                      0   
...                                             ...                    ...   
83434                                             0                      0   
83435                                             0                      0   
83436                                             1                      0   
83437                                             0                      0   
83438                                             0                      0   

                                                                            \
      Designer DevOps specialist Developer, QA or test Developer, back-end   
0            0                 0                     0                   0   
1            0                 0                     0                   0   
2            0                 0                     0                   0   
3            0                 0                     0                   0   
4            0                 0                     0                   0   
...        ...               ...                   ...                 ...   
83434        0                 0                     0                   1   
83435        0                 0                     0                   0   
83436        0                 1                     0                   0   
83437        0                 0                     0                   1   
83438        0                 0                     0                   1   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 0   
1                                                 0   
2                                                 0   
3                                                 0   
4                                                 1   
...                                             ...   
83434                                             0   
83435                                             0   
83436                                             0   
83437                                             0   
83438                                             0   

                                                   ...  \
      Developer, embedded applications or devices  ...   
0                                               0  ...   
1                                               0  ...   
2                                               0  ...   
3                                               0  ...   
4                                               0  ...   
...                                           ...  ...   
83434                                           0  ...   
83435                                          

In [8]:
# Look at the languages column to verify encoding process  
df['LanguageHaveWorkedWith'].T

,0,1,2,3,4,5,6,7,8,9,...,83429,83430,83431,83432,83433,83434,83435,83436,83437,83438
APL,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Assembly,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bash/Shell,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
C,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C#,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
C++,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
COBOL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Clojure,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
Crystal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dart,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


-------

## Display Jobs Frequency  

In [9]:
jobs_freq = df['DevType'].copy().sum().sort_values().reset_index()
jobs_freq

,index,0
0,Marketing or sales professional,638
1,Scientist,2015
2,"Senior Executive (C-Suite, VP, etc.)",2103
3,"Developer, game or graphics",2112
4,Educator,2369
5,"Engineer, site reliability",2448
6,Academic researcher,2899
7,Product manager,3074
8,Other (please specify):,3545
9,"Developer, QA or test",3611


--------

## Display Skills Frequency

In [10]:
# Calculate the sum for all cols except DevTypes
skills_freq = df.copy().drop('DevType', axis = 1).sum().reset_index()
skills_freq.columns = ['group' , 'skill' , 'freq']
sorted_freq_skills = skills_freq.sort_values('freq' ,ascending=False)
sorted_freq_skills

,group,skill,freq
171,ToolsTechHaveWorkedWith,Git,68171
185,ToolsTechWantToWorkWith,Git,58308
212,NEWCollabToolsHaveWorkedWith,Visual Studio Code,58026
19,LanguageHaveWorkedWith,JavaScript,53587
233,NEWCollabToolsWantToWorkWith,Visual Studio Code,48360
...,...,...,...
8,LanguageHaveWorkedWith,Crystal,466
6,LanguageHaveWorkedWith,COBOL,437
230,NEWCollabToolsWantToWorkWith,TextMate,396
173,ToolsTechHaveWorkedWith,Pulumi,368


In [11]:
fig = px.treemap(skills_freq , 
                 path= ['group', 'skill'],
                 color_continuous_scale= 'deep', 
                 values = 'freq' , color = 'freq')
fig.update_layout(width = 1400 , height = 700 )
fig.show()
fig.write_html(os.path.join(FIG_DIR , 'treemap_skills_freq.html'))

- `here from this treemap we can see that the most popular skills are in the data science field`

- `It's dynamic tree map we can interact with it and to know more about the skills by click to the treemap` 

--------

## Create Jobs & Skills Heatmap

In [12]:
sorted_roles = df['DevType'].sum().sort_values().index.tolist()
sorted_skills = df.drop('DevType' , axis = 1 ).sum().sort_values(ascending = False).droplevel(level = 0).index.tolist()

In [13]:
skills = []

# for each role, calculte the precentage of each skill  also filtering the role 
for role in sorted_roles:
    role_mask = (df[('DevType' , role)] == 1)
    # for each skill column , calculate mean of the one-hot-encoded -> precentage  and append to list 
    skills_role = pd.concat({tech_col: df.loc[role_mask , tech_col].mean() * 100 for tech_col in TECH_COLS})
    skills.append(skills_role)
# Concat 
skills  = pd.concat(skills , axis = 1)
skills.columns = sorted_roles
skills = skills.reset_index(level = 0 , drop= True)
skills = skills.loc[sorted_skills]
skills = skills.T

In [14]:
skills

,Git,Git,Git,Git,Visual Studio Code,Visual Studio Code,JavaScript,JavaScript,Visual Studio Code,Visual Studio Code,...,Crystal,Crystal,COBOL,COBOL,TextMate,TextMate,Pulumi,Pulumi,COBOL,COBOL
Marketing or sales professional,70.062696,58.150470,70.062696,58.150470,61.285266,50.626959,71.943574,55.015674,61.285266,50.626959,...,4.231975,4.231975,3.761755,3.134796,5.172414,4.388715,3.761755,3.761755,3.761755,3.134796
Scientist,81.687345,72.853598,81.687345,72.853598,52.109181,44.813896,42.431762,27.444169,52.109181,44.813896,...,1.588089,1.935484,1.637717,1.339950,2.630273,1.588089,0.942928,1.538462,1.637717,1.339950
"Senior Executive (C-Suite, VP, etc.)",84.688540,73.418925,84.688540,73.418925,66.096053,56.443176,77.223015,51.307656,66.096053,56.443176,...,1.854494,2.615311,2.092249,0.951022,3.185925,1.664289,2.234903,3.328578,2.092249,0.951022
"Developer, game or graphics",84.375000,72.632576,84.375000,72.632576,72.206439,60.037879,64.914773,40.198864,72.206439,60.037879,...,1.373106,1.515152,1.609848,1.089015,2.367424,1.515152,1.136364,1.609848,1.609848,1.089015
Educator,82.566484,71.718024,82.566484,71.718024,65.512875,54.537780,66.483748,47.783875,65.512875,54.537780,...,1.477417,1.646264,1.772900,1.266357,2.448290,1.519629,1.308569,1.899536,1.772900,1.266357
"Engineer, site reliability",89.011438,78.553922,89.011438,78.553922,67.606209,56.495098,64.174837,37.867647,67.606209,56.495098,...,1.633987,2.736928,1.552288,1.021242,2.165033,1.511438,2.900327,6.985294,1.552288,1.021242
Academic researcher,80.131080,70.196620,80.131080,70.196620,54.294584,46.257330,44.394619,29.354950,54.294584,46.257330,...,1.310797,1.379786,1.552259,1.000345,1.897206,1.345291,0.758882,1.103829,1.552259,1.000345
Product manager,83.702017,72.836695,83.702017,72.836695,70.364346,60.540013,75.829538,54.456734,70.364346,60.540013,...,1.919323,2.049447,1.626545,0.780742,2.179571,1.366298,1.073520,1.789200,1.626545,0.780742
Other (please specify):,75.994358,65.923836,75.994358,65.923836,61.100141,52.101551,57.122708,40.423131,61.100141,52.101551,...,1.015515,1.551481,1.156559,1.043724,1.354020,0.789845,0.818054,1.579690,1.156559,1.043724
"Developer, QA or test",86.901135,75.380781,86.901135,75.380781,69.426752,58.349488,72.445306,50.872335,69.426752,58.349488,...,1.384658,1.578510,1.301579,0.803102,1.661590,1.052340,1.024647,1.772362,1.301579,0.803102


In [16]:
fig = go.Figure(data= go.Heatmap(z = skills , x = skills.columns , y = skills.index , colorscale = 'magma' , ygap = 1))
fig.update_layout(width = 2500 , height = 700)
fig.show()
fig.write_html(os.path.join(FIG_DIR , 'heatmap_job_skills.html'))

--------

## Create Jobs dendrogram

In [ ]:
roles_short_dict = roles_names.set_index('Original name')['Short name '].to_dict()
short_labels = [roles_short_dict[role] for role in sorted_roles]

In [ ]:
fig = ff.create_dendrogram(skills , labels = short_labels , orientation= 'left' , color_threshold= 0)